In [39]:
# Install required libraries
!pip install mediapipe opencv-python

# Import libraries
import cv2
import mediapipe as mp
import threading
from threading import Thread
import tkinter as tk
from tkinter import messagebox


# Initialize MediaPipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

Defaulting to user installation because normal site-packages is not writeable


Thought process behind calibration:


Measure distance between users joints, and based on that calculate a threshold. 

In [40]:
# Function to calculate distance between two landmarks
def calculate_distance(landmarks, landmark_a, landmark_b):
    x_a, y_a = landmarks.landmark[landmark_a].x, landmarks.landmark[landmark_a].y
    x_b, y_b = landmarks.landmark[landmark_b].x, landmarks.landmark[landmark_b].y
    distance = ((x_b - x_a)**2 + (y_b - y_a)**2)**0.5
    return distance

# Function to display a message on the screen
def display_message(frame, message, duration):
    text_size = 1
    height, width, _ = frame.shape

    cv2.putText(frame, message, (int(width / 4), int(height / 2)),
                cv2.FONT_HERSHEY_SIMPLEX, text_size, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Hand Tracking', frame)
    cv2.waitKey(duration * 1000)

button_clicked = False



# Function to create and handle the calibration button
def create_calibration_button(frame):
    button_text = "Press me to begin calibration process"
    button_size = (650, 100)
    button_position = ((frame.shape[1] - button_size[0]) // 2, (frame.shape[0] - button_size[1]) // 2)

    # Create a black image with the button text
    button_image = frame.copy()
    cv2.rectangle(button_image, button_position, (button_position[0] + button_size[0], button_position[1] + button_size[1]), (0, 0, 0), cv2.FILLED)
    cv2.putText(button_image, button_text, (button_position[0] + 10, button_position[1] + 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    return button_image, button_position, button_size

# Mouse event callback function
def on_mouse_event(event, x, y, flags, param):
    global button_clicked, button_position, button_size

    if event == cv2.EVENT_LBUTTONDOWN:
        # Check if the click is inside the button's rectangle
        button_x, button_y, button_width, button_height = button_position[0], button_position[1], button_size[0], button_size[1]
        if button_x < x < button_x + button_width and button_y < y < button_y + button_height:
            button_clicked = True

# Display calibration button
def display_calibration_button(cap):
    global button_clicked, button_position, button_size

    cv2.namedWindow('Hand Tracking')
    cv2.setMouseCallback('Hand Tracking', on_mouse_event)

    while not button_clicked:
        ret, frame = cap.read()
        if not ret:
            break

        button_image, button_position, button_size = create_calibration_button(frame)  # Get button position and size

        cv2.imshow('Hand Tracking', button_image)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif cv2.getWindowProperty('Hand Tracking', cv2.WND_PROP_VISIBLE) < 1:
            break

    cv2.destroyAllWindows()


# Function to process webcam frames
def process_webcam():
    # Open the webcam
    cap = cv2.VideoCapture(0)  # 0 corresponds to the default webcam

    # Display calibration button
    display_calibration_button(cap)
    
    # Display calibration message
    ret, frame = cap.read()
    if ret:
        display_message(frame, "Calibration will begin in 5 seconds, please lay your hand flat and do not move", 5)


    # Initialize variables for calibration
    calibration_distance = 0
    is_calibrating = True
    calibration_duration = 3  # seconds

    current_distance = 0


    # Calibration start time
    calibration_start_time = cv2.getTickCount()

    # Calibration process
    while is_calibrating:
        # Read a frame from the webcam
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe hands
        results = hands.process(frame_rgb)

        # If hands are detected, draw landmarks on the frame
        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                # Draw landmarks
                mp.solutions.drawing_utils.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)

                # Calculate and store the distance during calibration
                calibration_distance = calculate_distance(landmarks,
                                                            mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value,
                                                            mp_hands.HandLandmark.MIDDLE_FINGER_PIP.value)
                
        # Display calibration message during the 5 seconds
        elapsed_time = (cv2.getTickCount() - calibration_start_time) / cv2.getTickFrequency()
        if elapsed_time < calibration_duration:
            display_message(frame, "Calibrating...", 1)

        # Calibration complete
        else:
            is_calibrating = False
            display_message(frame, "Calibration complete. You may now begin playing", 2)

    # Main loop for hand tracking after calibration
    while cap.isOpened():
        # Read a frame from the webcam
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe hands
        results = hands.process(frame_rgb)

        # If hands are detected, draw landmarks on the frame
        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                # Draw landmarks
                mp.solutions.drawing_utils.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)  

                # Get landmarks' coordinates
                for idx, landmark in enumerate(landmarks.landmark):
                    height, width, _ = frame.shape
                    cx, cy = int(landmark.x * width), int(landmark.y * height)

                    # Draw circles at the tips and proximal phalanges
                    if idx in [mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value,
                               mp_hands.HandLandmark.MIDDLE_FINGER_PIP.value]:
                        cv2.circle(frame, (cx, cy), 5, (255, 0, 0), cv2.FILLED)

                        #calculate current distance
                        current_distance = calculate_distance(landmarks,
                                                                        mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value,
                                                                        mp_hands.HandLandmark.MIDDLE_FINGER_PIP.value)

                        if current_distance > 0.9 * calibration_distance:
                            # Display a warning if the hand is flat
                            warning_text = "Hand is flat!"
                            text_size = 2  # Increase text size

                            # Display text at the top right corner
                            cv2.putText(frame, warning_text, (width - 600, 100), cv2.FONT_HERSHEY_SIMPLEX,
                                        text_size, (0, 0, 255), 2, cv2.LINE_AA)

        # Display the frame with landmarks
        cv2.imshow('Hand Tracking', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam capture object
    cap.release()
    cv2.destroyAllWindows()

process_webcam()


KeyboardInterrupt: 